# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd 
import numpy
from sqlalchemy import create_engine
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import nltk
import re
import string
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.grid_search import GridSearchCV

/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('DisasterResponseTable', engine)
X=df['message']
Y=df.iloc[:,4:]
#X = df.message.values
#Y = df.iloc[:, 4:].values

In [4]:
print(type(X))
print(type(Y))

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


In [5]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
 #"""define a function to remove numbers, remove white space, remove stop words, normalize words"""
    result = text.lower() #Convert text to lowercase
    result = re.sub(r'\d+', '', result) #Remove numbers
    result= "".join([char for char in result if char not in string.punctuation]) #Removing Punctuation
    result = result.strip() #remove white space

    stop_words = set(stopwords.words('english')) 
    match_tokenizer = RegexpTokenizer("[\w']+")
    match_tokens = match_tokenizer.tokenize(result)
    result = [i for i in match_tokens if not i in stop_words]  #Stop words removal

    lemmatizer=WordNetLemmatizer()    #reduce inflectional forms to a common base form.
    Lemmatization = [lemmatizer.lemmatize(word) for word in result]
    return Lemmatization
pass

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
#X, Y = make_multilabel_classification(n_classes=36, random_state=0)

In [8]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),
            ('clf', MultiOutputClassifier(RandomForestClassifier(n_jobs=-1)))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.25,random_state = 42)

In [10]:
print(X_train.shape)

(19662,)


In [11]:
pipeline.fit(X_train,Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
Y_pred=pipeline.predict(X_test)

In [13]:
#categories = Y.columns.tolist()
#print(classification_report(Y_test.values, Y_pred, target_names=categories))

In [14]:
Y_pred = pipeline.predict(X_test)
for i, col in enumerate(Y_test.columns): 
    print("category: ", col) 
    print(classification_report(Y_test.values[i], Y_pred[i]))

category:  related
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        35
          1       1.00      1.00      1.00         1

avg / total       1.00      1.00      1.00        36

category:  request
             precision    recall  f1-score   support

          0       0.97      1.00      0.99        35
          1       0.00      0.00      0.00         1

avg / total       0.95      0.97      0.96        36

category:  offer
             precision    recall  f1-score   support

          0       0.88      0.97      0.92        30
          1       0.67      0.33      0.44         6

avg / total       0.84      0.86      0.84        36

category:  aid_related
             precision    recall  f1-score   support

          0       1.00      0.97      0.99        36
          1       0.00      0.00      0.00         0

avg / total       1.00      0.97      0.99        36

category:  medical_help
             precision    recall  f1-s

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [15]:
parameters = {'clf__estimator__max_depth': [10, 50, None],
              'clf__estimator__min_samples_leaf':[2, 5, 10]}

cv = GridSearchCV(pipeline, parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
cv.fit(X_train,Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'clf__estimator__max_depth': [10, 50, None], 'clf__estimator__min_samples_leaf': [2, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [17]:
Y_pred = cv.predict(X_test)
for i, col in enumerate(Y_test.columns): 
    print("category: ", col) 
    print(classification_report(Y_test.values[i], Y_pred[i]))

category:  related
             precision    recall  f1-score   support

          0       0.97      1.00      0.99        35
          1       0.00      0.00      0.00         1

avg / total       0.95      0.97      0.96        36

category:  request
             precision    recall  f1-score   support

          0       0.97      1.00      0.99        35
          1       0.00      0.00      0.00         1

avg / total       0.95      0.97      0.96        36

category:  offer
             precision    recall  f1-score   support

          0       0.91      1.00      0.95        30
          1       1.00      0.50      0.67         6

avg / total       0.92      0.92      0.90        36

category:  aid_related
             precision    recall  f1-score   support

          0       1.00      0.97      0.99        36
          1       0.00      0.00      0.00         0

avg / total       1.00      0.97      0.99        36

category:  medical_help
             precision    recall  f1-s

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [18]:
print("\nBest Parameters:", cv.best_params_)


Best Parameters: {'clf__estimator__max_depth': None, 'clf__estimator__min_samples_leaf': 2}


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [19]:
#linear SVC
SVC_Pipeline =Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),
            ('multi_clf', MultiOutputClassifier(OneVsRestClassifier(LinearSVC())))])
SVC_Pipeline.fit(X_train,Y_train)
Y_pred=SVC_Pipeline.predict(X_test)
for i, col in enumerate(Y.columns): 
    print("category: ", col) 
    print(classification_report(Y_test.values[i], Y_pred[i]))

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


category:  related
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        35
          1       1.00      1.00      1.00         1

avg / total       1.00      1.00      1.00        36

category:  request
             precision    recall  f1-score   support

          0       0.97      1.00      0.99        35
          1       0.00      0.00      0.00         1

avg / total       0.95      0.97      0.96        36

category:  offer
             precision    recall  f1-score   support

          0       0.91      0.97      0.94        30
          1       0.75      0.50      0.60         6

avg / total       0.88      0.89      0.88        36

category:  aid_related
             precision    recall  f1-score   support

          0       1.00      0.94      0.97        36
          1       0.00      0.00      0.00         0

avg / total       1.00      0.94      0.97        36

category:  medical_help
             precision    recall  f1-s

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [20]:
#naive bayes
NB_Pipeline =Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),
            ('multi_clf', MultiOutputClassifier(OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None)))),
            ])
NB_Pipeline.fit(X_train,Y_train)
Y_pred=NB_Pipeline.predict(X_test)
for i, col in enumerate(Y.columns): 
    print("category: ", col) 
    print(classification_report(Y_test.values[i], Y_pred[i]))

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


category:  related
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        35
          1       1.00      1.00      1.00         1

avg / total       1.00      1.00      1.00        36

category:  request
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        35
          1       1.00      1.00      1.00         1

avg / total       1.00      1.00      1.00        36

category:  offer
             precision    recall  f1-score   support

          0       0.88      1.00      0.94        30
          1       1.00      0.33      0.50         6

avg / total       0.90      0.89      0.86        36

category:  aid_related
             precision    recall  f1-score   support

          0       1.00      0.97      0.99        36
          1       0.00      0.00      0.00         0

avg / total       1.00      0.97      0.99        36

category:  medical_help
             precision    recall  f1-s

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [21]:
#logistic regression
Log_Pipeline =Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),
            ('multi_clf', MultiOutputClassifier(OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=1))),
            ])
Log_Pipeline.fit(X_train,Y_train)
Y_pred=Log_Pipeline.predict(X_test)
for i, col in enumerate(Y.columns): 
    print("category: ", col) 
    print(classification_report(Y_test.values[i], Y_pred[i]))

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


category:  related
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        35
          1       1.00      1.00      1.00         1

avg / total       1.00      1.00      1.00        36

category:  request
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        35
          1       1.00      1.00      1.00         1

avg / total       1.00      1.00      1.00        36

category:  offer
             precision    recall  f1-score   support

          0       0.91      0.97      0.94        30
          1       0.75      0.50      0.60         6

avg / total       0.88      0.89      0.88        36

category:  aid_related
             precision    recall  f1-score   support

          0       1.00      0.97      0.99        36
          1       0.00      0.00      0.00         0

avg / total       1.00      0.97      0.99        36

category:  medical_help
             precision    recall  f1-s

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [22]:
with open('model.pkl', 'wb') as file:
    pickle.dump(cv, file)

NameError: name 'pickle' is not defined

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.